In [ ]:
from gaitlink.lr_detection._utils import _butter_lowpass_filter, _butter_bandpass_filter
# axis_config = "Vertical"
# axis_config = "Anterior-Posterior"
axis_config = "Combined"

window = 0

if axis_config == "Vertical":
    signal = data_list[window][:, 0]
elif axis_config == "Anterior-Posterior":
    signal = data_list[window][:, 2] * - 1 # invert the sign here
elif axis_config == "Combined":
    signal = (data_list[window][:, 2] * - 1) + data_list[window][:, 0]

if ic_list[window][-1] >= len(signal):
    print('edge case 1')
    ic_list[window][-1] = len(signal) - 1

if len(ic_list[window]) >= 2 and ic_list[window][-2] >= len(signal):
    print('edge case 2')
    ic_list[window][-2] = len(signal) - 2

sig = signal - np.mean(signal)
cutoff_freq = 2
sig_filtered = _butter_lowpass_filter(sig,
                                      cutoff= cutoff_freq,
                                      sampling_rate_hz = sampling_rate)
lower_band = 0.5
upper_band = 2
sig_filtered_2 = _butter_bandpass_filter(signal, 
                                        lower_bound = lower_band,
                                        upper_bound = upper_band,
                                        sampling_rate_hz = sampling_rate)

# import matplotlib.pyplot as plt
ic_left_right = np.where(sig_filtered_2[ic_list[window]] <= 0, "L", "R")
print(ic_left_right)

In [ ]:
# some improvements....
# from gaitlink.lr_detection._array_handling import find_extrema_in_radius
from gaitlink.lr_detection._utils import find_extrema_in_radius

version = 'snap_to_extremum'
# version = 'consistency_check'

data_list, walking_type_label, sequence_no,  ic_list, reference_lr_list = detector.extract_sequence_data(imu_data, event_list, reference_data = True)

plt.figure(figsize = (3.5, 10))
for window in range(len(event_list)):
    signal = data_list[window][:, 0]
    print(signal.shape)
    if ic_list[window][-1] >= len(signal):
        ic_list[window][-1] = len(signal) - 1
        
    lowpass_cut_off_hz = 2
    signal_filtered = _butter_lowpass_filter(signal,
                                            lowpass_cut_off_hz,
                                            sampling_rate) 

    # # TODO: fine tune these parameters, 30 & 30 seems to work just fine, but this does not seem ideal.
    radius_left = 30
    radius_right = 15
    plt.subplot(len(event_list), 1, window + 1)
    plt.plot(signal, 'r-')
    plt.plot(signal_filtered, 'b-')
    # ic_left_right = np.where(sig_filtered[ic_list[window]] <= 0, "L", "R")
    # print(ic_left_right)

    plt.plot(abs(signal_filtered), 'k--')
    
    version = 'snap_to_extremum'
    if version == "snap_to_extremum":
        # find extrema of absolute signal via function
        gyr_yaw_abs = abs(signal_filtered)

        extrema = find_extrema_in_radius(gyr_yaw_abs, ic_list[window], radius_left, radius_right, "max")
        ic_left_right_improved = np.where(signal_filtered[extrema]<=0, "L", "R")
        
        print(ic_left_right_improved)
        
        # ok, so this might not work with the current implementation, when you segment the data turn by turn... we might have to remove that feature..
    plt.vlines(ic_list[window][ic_left_right_improved == 'L'],
            ymin = ylimits[0],
            ymax = ylimits[1],
            color = 'red',
            linestyles = 'dashed',
            label = 'Left ICs' )

    plt.vlines(ic_list[window][ic_left_right_improved == 'R'],
            ymin = ylimits[0],
            ymax = ylimits[1],
            color = 'blue',
            linestyles = 'dashed',
            label = 'Right ICs')

In [ ]:
# inputs: imu_data
#         event_list
segment = 2
start = np.round(event_list[segment]['Start'] * sampling_rate).astype(int)
end = np.round(event_list[segment]['End'] * sampling_rate).astype(int)
ic_list = np.round(event_list[segment]['InitialContact_Event'] * sampling_rate).astype(int)
ic_list = ic_list - ic_list[0] # 0-indexed

# event_list[0]
sd_data = imu_data.loc[:, ['gyr_x', 'gyr_y', 'gyr_z']].to_numpy()[start : end, :]

In [ ]:
signal = sd_data[:, 0]

print(signal.shape)
if ic_list[-1] >= len(signal):
    ic_list[-1] = len(signal) - 1
    
lowpass_cut_off_hz = 2
signal_filtered = _butter_lowpass_filter(signal,
                                        lowpass_cut_off_hz,
                                        sampling_rate) 
radius_left = 30
radius_right = 15

version = 'snap_to_extremum'
if version == "snap_to_extremum":
     # find extrema of absolute signal via function
    gyr_yaw_abs = abs(signal_filtered)

    extrema = find_extrema_in_radius(gyr_yaw_abs, ic_list, radius_left, radius_right, "max")
    ic_left_right_improved = np.where(signal_filtered[extrema]<=0, "L", "R")
    
    print(ic_left_right_improved)
print(event_list[window]['InitialContact_LeftRight'])

In [ ]:
from gaitlink.lr_detection._utils import find_extrema_in_radius

version = 'consistency_check'

if version == "consistency_check":

    ic_left_right = np.where(signal_filtered[ic_list]<=0, "L", "R")
    ic_left_right_01 = np.where(signal_filtered[ic_list]<=0, 0, 1)

    # TODO idea: check where L R L R pattern in broken and only improve in those situations
    ic_left_right_01_diff = np.diff(ic_left_right_01)
    # the diff is 0 is two subsequent steps are assigned to the same foot
    consistency_check = np.where(ic_left_right_01_diff == 0)[0]
    print(consistency_check.size)
    # if consistency_check.size == 0:
    #     ic_left_right


    k = 1
    max_loops = 100
    
    while consistency_check.size != 0 and k < max_loops:
        # print(f'Loop {k}')
        # print(np.where(signal_filtered[ic_list_samples]<=0, 0, 1))
        # print(np.diff(np.where(signal_filtered[ic_list_samples]<=0, 0, 1)))
        # print("\n")
    
        # find extrema of absolute signal via function
        signal_abs = abs(signal_filtered)
    
        extrema = find_extrema_in_radius(signal_abs,
                                         ic_list,
                                         radius_left,
                                         radius_right,
                                         "max")
        
        ic_left_right_improved = np.where(signal_filtered[extrema] <= 0, "L", "R")
    
        ic_left_right_improved_01 = np.where(signal_filtered[extrema] <= 0, 0, 1)
        ic_left_right_improved_01_diff = np.diff(ic_left_right_improved_01)

        consistency_check = np.where(ic_left_right_improved_01_diff == 0)[0]
        
        k += 1
        
    # print(ic_left_right_improved)

In [ ]:
print(np.where(signal_filtered[ic_list]<=0, 0, 1))
print(np.diff(np.where(signal_filtered[ic_list]<=0, 0, 1)))

signal_abs = abs(signal_filtered)

extrema = find_extrema_in_radius(signal_abs,
                                ic_list,
                                radius_left,
                                radius_right,
                                "max")
print(extrema)
# this is always going to return the same thing, so there won't be any improvements...
# essentially, this is an infinite loop...
# ok, so just stick to the original implementation for now...

ic_left_right_improved = np.where(signal_filtered[extrema]<=0, "L", "R")

ic_left_right_improved_01 = np.where(signal_filtered[extrema]<=0, 0, 1)
ic_left_right_improved_01_diff = np.diff(ic_left_right_improved_01)

consistency_check = np.where(ic_left_right_improved_01_diff == 0)[0]

# print(consistency_check)

# this while loop is not going to improve anything...
